# Transformer (Translation)

In [1]:
import random
import numpy as np
import tensorflow as tf
from konlpy.tag import Okt

from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Lambda, Layer, Embedding, LayerNormalization

In [2]:
%pwd

'C:\\Users\\DCU\\Desktop\\Git-cum\\Fast_Cappus_-DeepLearning-AI\\5.딥러닝 최신 트렌드'

In [3]:
#import os
#os.chdir('/home/jeyong/Disks/hdd1/workspace/Jupyter/deeplearning_part3')

In [4]:
EPOCHS = 200
NUM_WORDS = 2000

## Dot-Scaled Attention

In [5]:
class DotScaledAttention(Layer):
    def __init__(self, d_emb, d_reduced, masked=False):
        super().__init__()
        self.q = Dense(d_reduced, input_shape=(-1, d_emb))
        self.k = Dense(d_reduced, input_shape=(-1, d_emb))
        self.v = Dense(d_reduced, input_shape=(-1, d_emb))
        self.scale = Lambda(lambda x: x/np.sqrt(d_reduced))
        self.masked = masked
        
    def call(self, x, training=None, mask=None): # (q,k,v)
        q = self.scale(self.q(x[0]))
        k = self.k(x[1])
        v = self.v(x[2])
        
        k_T = tf.transpose(k, perm=[0, 2, 1])
        comp = tf.matmul(q, k_T)
        
        if self.masked: # Referred from https://github.com/LastRemote/Transformer-TF2.0
            length = tf.shape(comp)[-1]
            mask = tf.fill((length, length), -np.inf)
            mask = tf.linalg.band_part(mask, 0, -1) # Get upper triangle
            mask = tf.linalg.set_diag(mask, tf.zeros((length))) # Set diagonal to zeros to avoid operations with infinity
            comp += mask
        comp = tf.nn.softmax(comp, axis=-1)
        return tf.matmul(comp, v)

## Multi-Head Attention

In [6]:
class MultiHeadAttention(Layer):
    def __init__(self, num_head, d_emb, d_reduced, masked=False):
        super().__init__()
        self.attention_list = list()
        for _ in range(num_head):
            self.attention_list.append(DotScaledAttention(d_emb, d_reduced, masked))
        self.linear = Dense(d_emb, input_shape=(-1, num_head * d_reduced))

    def call(self, x, training=None, mask=None):
        attention_list = [a(x) for a in self.attention_list]
        concat = tf.concat(attention_list, axis=-1)
        return self.linear(concat)

## Encoder

In [7]:
class Encoder(Layer):
    def __init__(self, num_head, d_reduced):
        super().__init__()
        self.num_head = num_head
        self.d_r = d_reduced
    
    def build(self, input_shape):       
        self.multi_attention = MultiHeadAttention(self.num_head, input_shape[-1], self.d_r)
        self.layer_norm1 = LayerNormalization(input_shape = input_shape)
        
        self.dense1 = Dense(input_shape[-1] * 4, input_shape = input_shape, activation = 'relu')
        self.dense2 = Dense(input_shape[-1],
                            input_shape=self.dense1.compute_output_shape(input_shape))
        self.layer_norm2 = LayerNormalization(input_shape=input_shape)
        super().build(input_shape)

    def call(self, x, training=None, mask=None):
        h = self.multi_attention((x, x, x))
        ln1 = self.layer_norm1(x + h)
        
        h = self.dense2(self.dense1(ln1))
        return self.layer_norm2(h + ln1)

    def compute_output_shape(self, input_shape):
        return input_shape

## Decoder

In [8]:
class Decoder(Layer):
    def __init__(self, num_head, d_reduced):
        super().__init__()
        self.num_head = num_head
        self.d_r = d_reduced
    
    def build(self, input_shape):
        self.self_attention = MultiHeadAttention(self.num_head, input_shape[0][-1], self.d_r, masked=True)
        self.layer_norm1 = LayerNormalization(input_shape = input_shape)
        
        self.multi_attention = MultiHeadAttention(self.num_head, input_shape[0][-1], self.d_r)
        self.layer_norm2 = LayerNormalization(input_shape = input_shape)
        
        self.dense1 = Dense(input_shape[0][-1] * 4, input_shape = input_shape[0], activation = 'relu')
        self.dense2 = Dense(input_shape[0][-1],
                            input_shape=self.dense1.compute_output_shape(input_shape[0]))
        self.layer_norm3 = LayerNormalization(input_shape=input_shape)

        super().build(input_shape)

    def call(self, inputs, training=None, mask=None): # (x, context)
        x, context = inputs
        h = self.self_attention((x, x, x))
        ln1 = self.layer_norm1(x + h)
        
        h = self.multi_attention((ln1, context, context))
        ln2 = self.layer_norm2(ln1 + h)
        
        h = self.dense2(self.dense1(ln2))
        return self.layer_norm3(h + ln2)

    def compute_output_shape(self, input_shape):
        return input_shape

## Positional Encoding

In [9]:
class PositionalEncoding(Layer):  # Referred from https://github.com/LastRemote/Transformer-TF2.0
    def __init__(self, max_len, d_emb): # 최대 길이를 알아야 위치 비율을 찾을 수 있다.
        super().__init__()
        self.sinusoidal_encoding = np.array([self.get_positional_angle(pos, d_emb) for pos in range(max_len)], dtype=np.float32)
        self.sinusoidal_encoding[:, 0::2] = np.sin(self.sinusoidal_encoding[:, 0::2])
        self.sinusoidal_encoding[:, 1::2] = np.cos(self.sinusoidal_encoding[:, 1::2])
        self.sinusoidal_encoding = tf.cast(self.sinusoidal_encoding, dtype=tf.float32)

    def call(self, x, training=None, mask=None):
        return x + self.sinusoidal_encoding[:tf.shape(x)[1]]

    def compute_output_shape(self, input_shape):
        return input_shape

    def get_angle(self, pos, dim, d_emb):
        return pos / np.power(10000, 2 * (dim // 2) / d_emb)

    def get_positional_angle(self, pos, d_emb):
        return [self.get_angle(pos, dim, d_emb) for dim in range(d_emb)]

## Transformer Architecture

In [10]:
class Transformer(Model):
    def __init__(self, src_vocab, dst_vocab, max_len, d_emb, d_reduced, n_enc_layer, n_dec_layer, num_head):
        super().__init__()
        self.enc_emb = Embedding(src_vocab, d_emb)
        self.dec_emb = Embedding(dst_vocab, d_emb)
        self.pos_enc = PositionalEncoding(max_len, d_emb)
        
        self.encoder = [Encoder(num_head, d_reduced) for _ in range(n_enc_layer)]
        self.decoder = [Decoder(num_head, d_reduced) for _ in range(n_dec_layer)]
        self.dense = Dense(dst_vocab, input_shape=(-1, d_emb))
        
    def call(self, inputs, training=None, mask=None): # (src_sentence, dst_sentence_shift)
        src_sent, dst_sent_shifted = inputs
        
        h_enc = self.pos_enc(self.enc_emb(src_sent))
        
        for enc in self.encoder:
            h_enc = enc(h_enc)
            
        h_dec = self.pos_enc(self.dec_emb(dst_sent_shifted))
        
        for dec in self.decoder:
            hdec = dec([h_dec, h_enc])
            
        return tf.nn.softmax(self.dense(h_dec), axis=-1)
        

## Dataset 준비

In [11]:
dataset_file = './chatbot_data.csv' # acquired from 'http://www.aihub.or.kr' and modified
okt = Okt()

with open(dataset_file, 'r', encoding='UTF8') as file:
    lines = file.readlines()
    seq = [' '.join(okt.morphs(line)) for line in lines]

questions = seq[::2]
answers = ['\t ' + lines for lines in seq[1::2]]

num_sample = len(questions)

perm = list(range(num_sample))
random.seed(0)
random.shuffle(perm)

train_q = list()
train_a = list()
test_q = list()
test_a = list()

for idx, qna in enumerate(zip(questions, answers)):
    q, a = qna
    if perm[idx] > num_sample//5:
        train_q.append(q)
        train_a.append(a)
    else:
        test_q.append(q)
        test_a.append(a)

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=NUM_WORDS,
                                                  filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~')

tokenizer.fit_on_texts(train_q + train_a)

train_q_seq = tokenizer.texts_to_sequences(train_q)
train_a_seq = tokenizer.texts_to_sequences(train_a)

test_q_seq = tokenizer.texts_to_sequences(test_q)
test_a_seq = tokenizer.texts_to_sequences(test_a)

x_train = tf.keras.preprocessing.sequence.pad_sequences(train_q_seq,
                                                        value=0,
                                                        padding='pre',
                                                        maxlen=64)
y_train = tf.keras.preprocessing.sequence.pad_sequences(train_a_seq,
                                                        value=0,
                                                        padding='post',
                                                        maxlen=65)
y_train_shifted = np.concatenate([np.zeros((y_train.shape[0], 1)), y_train[:, 1:]], axis=1)


x_test = tf.keras.preprocessing.sequence.pad_sequences(test_q_seq,
                                                       value=0,
                                                       padding='pre',
                                                       maxlen=64)
y_test = tf.keras.preprocessing.sequence.pad_sequences(test_a_seq,
                                                       value=0,
                                                       padding='post',
                                                       maxlen=65)

## Train using keras

In [12]:
transformer = Transformer(NUM_WORDS, NUM_WORDS, 128, 16, 16, 2, 2, 4) # Instantiating a new transformer model

transformer.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
transformer.fit([x_train, y_train_shifted], y_train, batch_size=5, epochs=EPOCHS)

Epoch 1/200


80/80 [==============================] - 1s 11ms/step - loss: 7.0670 - accuracy: 0.2718
Epoch 2/200
80/80 [==============================] - 1s 7ms/step - loss: 5.5249 - accuracy: 0.6623
Epoch 3/200
80/80 [==============================] - 1s 7ms/step - loss: 3.3308 - accuracy: 0.8448
Epoch 4/200
80/80 [==============================] - 1s 7ms/step - loss: 1.4766 - accuracy: 0.9039
Epoch 5/200
80/80 [==============================] - 1s 7ms/step - loss: 0.8510 - accuracy: 0.9087
Epoch 6/200
80/80 [==============================] - 1s 7ms/step - loss: 0.6735 - accuracy: 0.9110
Epoch 7/200
80/80 [==============================] - 1s 7ms/step - loss: 0.5939 - accuracy: 0.9110
Epoch 8/200
80/80 [==============================] - 1s 7ms/step - loss: 0.5463 - accuracy: 0.9112
Epoch 9/200
80/80 [==============================] - 1s 8ms/step - loss: 0.5127 - accuracy: 0.9142
Epoch 10/200
80/80 [==============================] - 1s 7ms/step - loss: 0.4867 - accuracy: 0.9169
Epoch 11/200
80/80 [

80/80 [==============================] - 1s 8ms/step - loss: 0.0443 - accuracy: 0.9954
Epoch 84/200
80/80 [==============================] - 1s 8ms/step - loss: 0.0431 - accuracy: 0.9958
Epoch 85/200
80/80 [==============================] - 1s 8ms/step - loss: 0.0418 - accuracy: 0.9962
Epoch 86/200
80/80 [==============================] - 1s 7ms/step - loss: 0.0406 - accuracy: 0.9966
Epoch 87/200
80/80 [==============================] - 1s 7ms/step - loss: 0.0395 - accuracy: 0.9971
Epoch 88/200
80/80 [==============================] - 1s 7ms/step - loss: 0.0383 - accuracy: 0.9973
Epoch 89/200
80/80 [==============================] - 1s 7ms/step - loss: 0.0372 - accuracy: 0.9978
Epoch 90/200
80/80 [==============================] - 1s 8ms/step - loss: 0.0362 - accuracy: 0.9980
Epoch 91/200
80/80 [==============================] - 1s 7ms/step - loss: 0.0351 - accuracy: 0.9981
Epoch 92/200
80/80 [==============================] - 1s 7ms/step - loss: 0.0341 - accuracy: 0.9985
Epoch 93/200


80/80 [==============================] - 1s 9ms/step - loss: 0.0017 - accuracy: 1.0000
Epoch 165/200
80/80 [==============================] - 1s 8ms/step - loss: 0.0016 - accuracy: 1.0000
Epoch 166/200
80/80 [==============================] - 1s 8ms/step - loss: 0.0016 - accuracy: 1.0000
Epoch 167/200
80/80 [==============================] - 1s 7ms/step - loss: 0.0015 - accuracy: 1.0000
Epoch 168/200
80/80 [==============================] - 1s 7ms/step - loss: 0.0014 - accuracy: 1.0000
Epoch 169/200
80/80 [==============================] - 1s 6ms/step - loss: 0.0014 - accuracy: 1.0000
Epoch 170/200
80/80 [==============================] - 1s 7ms/step - loss: 0.0013 - accuracy: 1.0000
Epoch 171/200
80/80 [==============================] - 1s 7ms/step - loss: 0.0013 - accuracy: 1.0000
Epoch 172/200
80/80 [==============================] - 1s 7ms/step - loss: 0.0012 - accuracy: 1.0000
Epoch 173/200
80/80 [==============================] - 1s 6ms/step - loss: 0.0012 - accuracy: 1.0000
Epoc